In [112]:
import math
import datetime
import numpy as np
import pandas as pd
from datascience import *
from linearmodels.panel import FamaMacBeth, PanelOLS


# I. Import Data

In [129]:
data = Table.read_table("../data/simulated_data.csv") # headers pre-added
data.sort("date")

id,ticker,date,time,Old,ClosestNeighbor,length,closest1,closest2
8TAW6K50XY,9769Z,20150223,16:06:32,0.63,0.14,288,YOMDOKHRFD,NAAQUDSJRC
8TAW6K50XY,SF,20150223,16:06:32,0.07,0.02,343,PQJNIPIJUS,TTFNYRKMNP
A7186S972M,BA,20150224,10:00:44,0.33,0.05,407,UTBGWCGZPC,VDFQEFZZKG
AD676TTDS4,BAC,20150224,12:13:19,0.63,0.11,271,RACFHISWOW,TPNEHGLCJC
9VXX6K50YG,HD,20150224,06:01:09,0.84,0.52,400,VKPCWAAXBQ,HJEMPDYORB
9YP56K50Y9,CMCSA,20150224,07:00:41,0.91,0.45,135,BUBWPAFEYK,SUYDJOSSPP
A70T6S972G,BA,20150224,10:00:29,0.83,0.21,234,YSETNVDMAF,GKDYFAZNVT
ANXC6K50XX,HPQ,20150224,16:05:36,0.38,0.02,420,ZDGHORHGFV,WQZIVSCZWU
C7Z16KLVRE,JPM,20150225,12:16:13,0.16,0.04,98,PSPYXMGDZL,GXOBQGIXJK
CFNR6S972O,HPQ,20150225,15:02:15,0.15,0.03,178,GIMZXBSEWX,ZYOHBXFJXO


In [114]:
quarterly = Table.read_table("../data/quarterly.csv")

In [115]:
quarterly = quarterly.with_column("atqr_exists", np.isnan(quarterly.column("atqr")))
quarterly = quarterly.where("atqr_exists", False).drop("atqr_exists")
quarterly = quarterly.select("tic","datadate","atqr").relabel("tic", "ticker").relabel("datadate", "date").relabel("atqr","BVal")
quarterly.sort("date")

ticker,date,BVal
AIR,19870228,228.106
AELNA,19870228,90.447
4225B,19870228,5.614
7547B,19870228,19.135
CECE,19870228,1.084
ASA,19870228,747.064
ADDC,19870228,2.505
ADVC.,19870228,42.461
AESM,19870228,23.839
ALAN,19870228,11.022


In [116]:
daily = Table.read_table("../data/daily.csv") # TODO: Only 2015??

In [ ]:
daily = daily.with_column("MCap", daily.column("cshoc") * daily.column("prcod"))
daily = daily.with_column("return", (daily.column("prcod") - daily.column("prccd"))/daily.column("prcod"))
daily = daily.with_column("frac", daily.column("cshtrd")/daily.column("cshoc"))
daily = daily.with_column("volatility", daily.column("prchd") - daily.column("prcld"))

In [118]:
# ERROR: Data N/A?
daily = daily.with_column("MCap_exists", np.isnan(daily.column("MCap")))
daily = daily.where("MCap_exists", False).drop("MCap_exists")

In [134]:
daily = daily.drop("gvkey", "iid").relabel("datadate", "date").relabel("tic", "ticker")

In [136]:
daily.sort('date')

date,ticker,cshoc,cshtrd,prccd,prchd,prcld,prcod,MCap,return,frac,volatility
20150102,AIR,3.9791e+07,153348,27.21,27.84,26.79,27.8,1.10619e+09,0.021223,0.00385384,1.05
20150102,AAL,7.17264e+08,1.07496e+07,53.91,54.6,53.07,54.28,3.89331e+10,0.00681651,0.014987,1.53
20150102,CECE,2.5866e+07,25996,15.38,15.69,15.0691,15.69,4.05838e+08,0.0197578,0.00100503,0.6209
20150102,ASA,1.929e+07,82894,10.42,10.42,9.98,9.98,1.92514e+08,-0.0440882,0.00429725,0.44
20150102,AVX,1.67957e+08,127014,13.87,14.1,13.7555,14.03,2.35644e+09,0.0114041,0.000756229,0.3445
20150102,PNW,1.1045e+08,640202,68.64,68.83,67.69,68.58,7.57466e+09,-0.000874891,0.00579631,1.14
20150102,AAN,7.2474e+07,898851,30.62,30.86,30.04,30.81,2.23292e+09,0.00616683,0.0124024,0.82
20150102,ABT,1.50579e+09,3.21656e+06,44.9,45.4501,44.635,45.25,6.8137e+10,0.00773481,0.00213613,0.8151
20150102,WDDD,9.6852e+07,53437,0.142,0.149,0.14,0.148,1.43341e+07,0.0405405,0.000551739,0.009
20150102,ACET,2.9065e+07,158985,21.62,21.964,21.215,21.89,6.36233e+08,0.0123344,0.00546998,0.749


# II. Individual Story Factors

In [120]:
OLD_THRESHOLD = 0.6
REPRINT_RECOMBINATION_THRESHOLD = 0.8

In [130]:
data = data.with_column("OldNews", data.column("Old") > OLD_THRESHOLD)
data = data.with_column("share_spanned", data.column("ClosestNeighbor") / data.column("Old"))
data = data.with_column("Reprint", data.apply(lambda row: row[9] and row[10] >= REPRINT_RECOMBINATION_THRESHOLD))
data = data.with_column("Recombination", data.apply(lambda row: row[9] and row[10] < REPRINT_RECOMBINATION_THRESHOLD))
data = data.relabel("length", "unique")

In [131]:
data

id,ticker,date,time,Old,ClosestNeighbor,unique,closest1,closest2,OldNews,share_spanned,Reprint,Recombination
A7186S972M,BA,20150224,10:00:44,0.33,0.05,407,UTBGWCGZPC,VDFQEFZZKG,False,0.151515,False,False
AD676TTDS4,BAC,20150224,12:13:19,0.63,0.11,271,RACFHISWOW,TPNEHGLCJC,True,0.174603,False,True
C7Z16KLVRE,JPM,20150225,12:16:13,0.16,0.04,98,PSPYXMGDZL,GXOBQGIXJK,False,0.25,False,False
CFNR6S972O,HPQ,20150225,15:02:15,0.15,0.03,178,GIMZXBSEWX,ZYOHBXFJXO,False,0.2,False,False
E4ON6JIJUR,CMCSA,20150226,13:00:23,0.1,0.02,298,IZBWTBNLUP,JHXAECOFVL,False,0.2,False,False
E4ON6JIJUR,VZ,20150226,13:00:23,0.29,0.27,160,EUNHUPLRKF,YCLFBIGDMC,False,0.931034,False,False
FWPF6TTDSE,KKR,20150227,12:03:15,0.71,0.52,59,DONFCGLDHG,KDNMXAGHGT,True,0.732394,False,True
8TAW6K50XY,9769Z,20150223,16:06:32,0.63,0.14,288,YOMDOKHRFD,NAAQUDSJRC,True,0.222222,False,True
8TAW6K50XY,SF,20150223,16:06:32,0.07,0.02,343,PQJNIPIJUS,TTFNYRKMNP,False,0.285714,False,False
9VXX6K50YG,HD,20150224,06:01:09,0.84,0.52,400,VKPCWAAXBQ,HJEMPDYORB,True,0.619048,False,True


# III. Firm Factors

In [122]:
groupCount = data.group(["ticker", "date"])
groupSum = data.group(["ticker", "date"], np.sum)

In [123]:
firms = Table().with_columns("ticker", groupCount.column("ticker"),
                             "date", groupCount.column("date"),
                             "PctOld", groupSum.column("OldNews sum")/groupCount.column("count"),
                             "PctRecombination", groupSum.column("Recombination sum")/groupCount.column("count"),
                             "ExtentOld", groupSum.column("Old sum")/groupCount.column("count"),
                             "ExtentRecombination", (groupSum.column("Old sum")-groupSum.column("ClosestNeighbor sum"))/groupCount.column("count"))

In [124]:
avg_unique_data = data.group(["date"], np.average)
def avg_unique(date):
    return avg_unique_data.where("date", date).column("unique average").item(0)

### Abnormal Factors

In [125]:
firms = firms.with_columns("log(|S|)", np.log(groupCount.column("count")),
                          "log(avg_unique)", np.log(firms.apply(avg_unique, "date")),
                          "log(avg_unique)^2", np.square(np.log(firms.apply(avg_unique, "date"))))

In [126]:
firms_df = firms.to_df().set_index(['ticker', 'date'])

extentOldModel = PanelOLS(firms_df[["ExtentOld"]], firms_df[["log(|S|)", "log(avg_unique)", "log(avg_unique)^2"]]).fit()
extentRecombinationModel = PanelOLS(firms_df[["ExtentRecombination"]], firms_df[["log(|S|)", "log(avg_unique)", "log(avg_unique)^2"]]).fit()

abnPctOld = Table.from_df(firms_df).column("ExtentOld") - Table.from_df(extentOldModel.predict(firms_df[["log(|S|)", "log(avg_unique)", "log(avg_unique)^2"]])).column("predictions")
abnPctRecombination = Table.from_df(firms_df).column("ExtentOld") - Table.from_df(extentRecombinationModel.predict(firms_df[["log(|S|)", "log(avg_unique)", "log(avg_unique)^2"]])).column("predictions")

firms = firms.with_columns("AbnPctOld", abnPctOld,
                           "AbnPctRecombination", abnPctRecombination)

In [127]:
firms

ticker,date,PctOld,PctRecombination,ExtentOld,ExtentRecombination,log(|S|),log(avg_unique),log(avg_unique)^2,abnPctOld,abnPctRecombination
0084207D,20150619,0,0,0.19,0.01,0,4.77576,22.8079,-0.345976,-0.0837963
0196476D,20150331,0.5,0.5,0.675,0.26,0.693147,5.67091,32.1593,0.16177,0.437619
0604939D,20150306,0,0,0.1,0.08,0,5.02607,25.2614,-0.438796,-0.167215
0953834D,20150423,1,1,0.92,0.68,0,5.64696,31.8882,0.385077,0.678109
0966904D,20150520,1,1,1.01,0.47,0,5.54796,30.7799,0.473421,0.763212
1012005D,20150331,0,0,0.42,0.05,0,5.67091,32.1593,-0.114464,0.179343
1013340Z,20150611,1,1,0.73,0.25,0,5.35517,28.6778,0.191328,0.47461
1254673D,20150526,0,0,0.24,0.15,0,5.64191,31.8311,-0.295017,-0.00214864
129626Z,20150310,0,0,0.06,0.04,0,5.59099,31.2591,-0.475908,-0.1847
1313606D,20150407,0,0,0.07,-0.01,0,5.74833,33.0433,-0.46282,-0.166538


# IV. Regressions

In [149]:
def dateSubtract(d, minusDays):
    new_date = datetime.date(d//10000, d//100%100, d%100) - datetime.timedelta(minusDays)
    return new_date.year*10000+new_date.month*100+new_date.day

In [ ]:
# ERROR: Ticker DNE?
tickerDNE = []
# """
for ticker in firms.group("ticker").column("ticker"):
    if daily.where("ticker", ticker).num_rows == 0:
        tickerDNE.append(ticker)
# """

In [ ]:
dailyAverage = daily.group("date", np.average)

# CHECK: Calculating 'return'? assuming: (close - open)/open (%)
# CHECK: value-weighted?
def abnRet(row):
    if row[0] in tickerDNE:
        return None
    prevDayAvgReturn = dailyAverage.where("date", dateSubtract(row[1], 1)).column("return average").item(0)
    return daily.where("ticker", row[0]).where("date", row[1]).column("return").item(0) - prevDayAvgReturn

# CHECK: 'shares turned over' = trading volume?
# assuming: frac for day t+1, avg frac for all firms on day t
def abnVol(row):
    if row[0] in tickerDNE:
        return None
    prevDayAvgFrac = dailyAverage.where("date", dateSubtract(row[1], 1)).column("frac average").item(0)
    return daily.where("ticker", row[0]).where("date", row[1]).column("frac").item(0) - prevDayAvgFrac

firms = firms.with_column("AbnRet", groupCount.apply(abnRet))
firms = firms.with_column("AbnVol", groupCount.apply(abnVol))

In [ ]:
firms # check to this point, then del

In [ ]:
def abn_stories(row):
    forFirm = groupCount.where("ticker", row[0])
    oneWeekAgo = dateSubtract(row[1], 5)
    twoMonthsAgo = dateSubtract(row[1], 60)
    pastWeek = np.average(forFirm.where("date", are.between(oneWeekAgo, row[1])).column("count"))
    pastTwoMonths = np.average(forFirm.where("date", are.between(twoMonthsAgo, oneWeekAgo)).column("count"))
    return pastWeek - pastTwoMonths

def m_cap(row):
    return daily.where("ticker", row[0]).where("date", row[1]).column("MCap").item(0)

def recent_quarter_bval(row):
    return quarterly.where("ticker", row[0]).where("date", are.below_or_equal_to(row[1])).sort("date").column("BVal").item(0)

def bm(row):
    return recent_quarter_bval(row)/m_cap(row)

def abn_ret_pweek(row):
    return np.sum(firms.where("ticker", row[0]).where("date", are.between(dateSubtract(row[1], 5), row[1])).column("AbnRet"))

def abn_vol_pweek(row):
    return np.average(firms.where("ticker", row[0]).where("date", are.between(dateSubtract(row[1], 5), row[1])).column("AbnVol"))

# CHECK: calculating volatility (high-low)
def abn_volatility_pweek(row):
    prevDaysAvgVolatility = dailyAverage.where("date", are.between(dateSubtract(row[1], 5), row[1])).column("frac volatility").item(0)
    return daily.where("ticker", row[0]).where("date", row[1]).column("volatility").item(0) - prevDayAvgVolatility

# CHECK: volume = closing * trading vol
def illiq_pweek(row):
    prior_week = daily.where("ticker", row[0]).where("date", are.between(dateSubtract(row[1], 5), row[1]))
    prior_week_illiq = 10e6 * prior_week.column("return") / (prior_week.column("cshtrd") * prior_week.column("prccd"))
    return np.log(np.average(prior_week_illiq))

Stories = groupCount.column("count") # all stories in data are relevant, each row is a story
AbnStories = groupCount.apply(abn_stories)
Terms = groupSum.column("unique sum")/groupCount.column("count")
MCap = groupCount.apply(m_cap)
BM = groupCount.apply(bm)
# TODO: business days?
AbnRetPW = groupCount.apply(abn_ret_pweek)
AbnVolPW = groupCount.apply(abn_vol_pweek)
AbnVolatilityPW = groupCount.apply(abn_volatility_pweek)
IlliqPW = groupCount.apply(illiq_pweek)

In [ ]:
firms = firms.with_columns("Stories", Stories,
                           "AbnStories", AbnStories,
                           "Terms", Terms,
                           "MCap", MCap,
                           "BM", BM,
                           "AbnRetPW", AbnRetPW,
                           "AbnVolPW", AbnVolPW,
                           "AbnVolatilityPW", AbnVolatilityPW,
                           "IlliqPW", IlliqPW)

In [ ]:
firms

In [ ]:
firms = firms.with_column("a", 1)
firms_df = firms.take().to_df().set_index(['ticker', 'date'])

### Regressions for Market Reactions to Old News

In [126]:
params = ["a", "AbnPctOld", "Stories", "AbnStories", "Terms", "MCap", "BM", "AbnRetPW", "AbnVolPW", "AbnVolatilityPW", "IlliqPW"]

# Do we need to shift AbnRet and AbnVol by a day? Yes.
abnRetModel_OldNews = FamaMacBeth(firms_df[["AbnRet"]][1:], firms_df[params][:-1]).fit('heteroskedastic', 'bartlett')
abnVolModel_OldNews = FamaMacBeth(firms_df[["AbnVol"]][1:], firms_df[params][:-1]).fit('heteroskedastic', 'bartlett')

print(abnRetModel_OldNews, abnVolModel_OldNews)

### Regressions for Market Reactions to Recombinations

In [ ]:
params = ["a", "AbnPctOld", "AbnPctRecombinations", "Stories", "AbnStories", "Terms", "MCap", "BM", "AbnRetPW", "AbnVolPW", "AbnVolatilityPW", "IlliqPW"]

abnRetModel_Recombination = FamaMacBeth(firms_df[["AbnRet"]][1:], firms_df[params][:-1]).fit('heteroskedastic', 'bartlett')
abnVolModel_Recombination = FamaMacBeth(firms_df[["AbnVol"]][1:], firms_df[params][:-1]).fit('heteroskedastic', 'bartlett')

print(abnRetModel_Recombination, abnVolModel_Recombination)

### Regressions for Market Reversal Reactions to Recombinations

In [ ]:
firms = firms.with_columns("AbnPctOld*AbnRet", firms.column("AbnPctOld")*firms.column("AbnRet"),
                           "AbnPctRecombination*AbnRet", firms.column("AbnPctRecombination")*firms.column("AbnRet"))
firms_df = firms.take().to_df().set_index(['ticker', 'date'])

In [ ]:
# CHECK: Typo on pg21, last paragraph?
params = ["a", "AbnPctOld", "AbnPctOld*AbnRet", "AbnRet", "AbnPctRecombination", "AbnPctRecombination*AbnRet", "Stories", "AbnStories", "Terms", "MCap", "BM", "AbnRetPW", "AbnVolPW", "AbnVolatilityPW", "IlliqPW"]
# THEN: Switch this to [t+t1, t+t2] instead of just [t+1]
abnRetModel_Reversal = FamaMacBeth(firms_df[["AbnRet"]][1:], firms_df[params][:-1]).fit('heteroskedastic', 'bartlett')
print(abnRetModel_Reversal)